In [2]:
import transformers
from accelerate import init_empty_weights
from transformers import AutoConfig, AutoModelForCausalLM

In [7]:
from tqdm import tqdm
import torch

total_size = 0
index_map = {"weight_map": {}}
for i in range(36):
    ckpt_path = f"/home/peiran/FMproj/output/global_step10000/layer_{i:02d}-model_states.pt"
    model_states = torch.load(ckpt_path, map_location='cpu')
    print(ckpt_path)
    all_keys = list(model_states.keys())
    print(all_keys)
#     for key in all_keys:
#         weight = model_states.pop(key)
#         if key.startswith("embed_tokens.weight") or key.startswith("mol_rep_layernorm") or key.startswith("mol_adapter"):
#             key = "model.molecule_embedding." + key
#         elif key == "weight":
#             key = "model.norm." + key
#         elif key == "lm_head.weight":
#             key = "llama_lm_head_and_loss." + key
#         else:
#             assert i >= 1 and i < 61
#             key = f"model.layers.{i - 1}." + key
#         index_map["weight_map"][key] = f"layer_{i:02d}-model_states.bin"
#         model_states[key] = weight
#         total_size += weight.nelement() * weight.element_size()
#     torch.save(model_states, f"/mnt/shiyu/models/converted/30B-mix-smi-pubmed-c4/global_step170000/layer_{i:02d}-model_states.bin")
# index_map["total_size"] = total_size

/home/peiran/FMproj/output/global_step10000/layer_00-model_states.pt
['dummy', 'graph_node_feature.atom_encoder.weight', 'graph_node_feature.in_degree_encoder.weight', 'graph_node_feature.out_degree_encoder.weight', 'graph_node_feature.graph_token.weight', 'graph_node_feature.atom_mask_embedding.weight', 'graph_attn_bias.edge_encoder.weight', 'graph_attn_bias.edge_dis_encoder.weight', 'graph_attn_bias.spatial_pos_encoder.weight', 'graph_attn_bias.graph_token_virtual_distance.weight', 'emb_layer_norm.weight', 'emb_layer_norm.bias', 'layers.0.self_attn.k_proj.weight', 'layers.0.self_attn.v_proj.weight', 'layers.0.self_attn.v_proj.bias', 'layers.0.self_attn.q_proj.weight', 'layers.0.self_attn.q_proj.bias', 'layers.0.self_attn.out_proj.weight', 'layers.0.self_attn.out_proj.bias', 'layers.0.self_attn.layer_norm.weight', 'layers.0.self_attn.layer_norm.bias', 'layers.0.self_attn_layer_norm.weight', 'layers.0.self_attn_layer_norm.bias', 'layers.0.fc1.weight', 'layers.0.fc1.bias', 'layers.0.fc2

In [10]:
config = AutoConfig.from_pretrained("/mnt/shiyu/models/converted/30B-mix-smi-pubmed-c4/global_step170000/",
            cache_dir=False,
            load_in_8bit=False,
            device_map=None,
            molrep_dict_path="/mnt/shiyu/dataset/moleculenet-data/processed/feature2d/merged-feature2d/",
            mfm_hidden_size=6144,
            use_pp=False,
            return_dict=False)
with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config)

In [11]:
model.tie_weights()

In [12]:
from accelerate import load_checkpoint_and_dispatch

model = load_checkpoint_and_dispatch(
    model, "/mnt/shiyu/models/converted/30B-mix-smi-pubmed-c4/global_step170000/", device_map="auto", no_split_module_classes=["LlamaDecoderLayer"]
)

{'model.molecule_embedding': 0, 'model.layers.0': 0, 'model.layers.1': 0, 'model.layers.2': 0, 'model.layers.3': 0, 'model.layers.4': 0, 'model.layers.5': 0, 'model.layers.6': 0, 'model.layers.7': 1, 'model.layers.8': 1, 'model.layers.9': 1, 'model.layers.10': 1, 'model.layers.11': 1, 'model.layers.12': 1, 'model.layers.13': 1, 'model.layers.14': 1, 'model.layers.15': 2, 'model.layers.16': 2, 'model.layers.17': 2, 'model.layers.18': 2, 'model.layers.19': 2, 'model.layers.20': 2, 'model.layers.21': 2, 'model.layers.22': 2, 'model.layers.23': 3, 'model.layers.24': 3, 'model.layers.25': 3, 'model.layers.26': 3, 'model.layers.27': 3, 'model.layers.28': 3, 'model.layers.29': 3, 'model.layers.30': 3, 'model.layers.31': 4, 'model.layers.32': 4, 'model.layers.33': 4, 'model.layers.34': 4, 'model.layers.35': 4, 'model.layers.36': 4, 'model.layers.37': 4, 'model.layers.38': 4, 'model.layers.39': 5, 'model.layers.40': 5, 'model.layers.41': 5, 'model.layers.42': 5, 'model.layers.43': 5, 'model.lay

In [24]:
import json

with open("/mnt/shiyu/models/converted/30B-mix-smi-pubmed-c4/global_step170000/pytorch_model.bin.index.json", "w") as out_file:
    json.dump(index_map, out_file)

In [13]:
from train import DEFAULT_PAD_TOKEN, DEFAULT_EOS_TOKEN, DEFAULT_BOS_TOKEN, DEFAULT_UNK_TOKEN, smart_tokenizer_and_embedding_resize

tokenizer = transformers.AutoTokenizer.from_pretrained(
    "/mnt/shiyu/models/converted/30B-mix-smi-pubmed-c4/global_step170000/",
    cache_dir=False,
    model_max_length=512,
    padding_side="right",
    use_fast=False,
)
special_tokens_dict = dict()
if tokenizer.pad_token is None:
    special_tokens_dict["pad_token"] = DEFAULT_PAD_TOKEN
if tokenizer.eos_token is None:
    special_tokens_dict["eos_token"] = DEFAULT_EOS_TOKEN
if tokenizer.bos_token is None:
    special_tokens_dict["bos_token"] = DEFAULT_BOS_TOKEN
if tokenizer.unk_token is None:
    special_tokens_dict["unk_token"] = DEFAULT_UNK_TOKEN

smart_tokenizer_and_embedding_resize(
    special_tokens_dict=special_tokens_dict,
    tokenizer=tokenizer,
    model=model,
    use_pp=False
)

Using pad_token, but it is not set yet.


In [46]:
tokenized = tokenizer("To fix a broken audio device in my laptop",
            return_tensors="pt",
            padding="longest",
            max_length=512,
            truncation=True)
res = model.generate(tokenized.input_ids[0].unsqueeze(0), do_sample=True, temperature=0.7, max_new_tokens=128, output_scores=True, return_dict_in_generate=True)

In [47]:
tokenizer.decode(res.sequences[0], skip_special_tokens=True)

'To fix a broken audio device in my laptop, I have to use the following command to fix this issue. This is based on the following command to fix this issue. After that, to confirm the changes, you can use this other command. '